In [6]:
#nbi:hide_in
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.optimize import curve_fit
import matplotlib.gridspec as gridspec
#from __future__ import print_function
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import pylab as pl

In [39]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Interactive run: test

In [43]:
#nbi:hide_in
def h():

        Stochasticity=1;
        N0 = np.zeros((2,100,100));
        N0[0,:50, :30]+=10**(4-Stochasticity);
        N0[1,50:, :30]+=10**(4-Stochasticity);
        test = Sim2Ds(N0=N0, K=10**(4-Stochasticity), Tmax=500, fs=0);
        test.generate_step(param = (1., 1., 0., 0.) );
        test.proceed();
        a = analysis(test);
        image = np.zeros( (10,100,100,3) )
        for i in np.arange(10):
            image[i] = a.snapsave(t=i)
        fig, ax = plt.subplots(2,5, figsize=(20,10) )
        fig.subplots_adjust(hspace=0. )
        for i in np.arange(10):
            ax[int(i/5),i%5 ].imshow(image[i])
            ax[int(i/5),i%5 ].set_xticks([]);
            ax[int(i/5),i%5 ].set_yticks([]);


w = interactive(h, {'manual': True})#, Stochasticity=widgets.IntSlider(min=1, max=4,value=1) ); # , time_range=widgets.IntRangeSlider(min=0, max=9, step=1, value=[0,9]) );
display(w)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [8]:
#nbi:hide_in
######################## Code for simulation

In [28]:
class Sim2Ds:
    '''  2D simulation, stochastic, simple'''
    def __init__(self, L = 100, Tmax = 1000, N0 = [], Ts = 0, dt=0.1, K=1000, model=0, fs=0.05, Filename = 'temp.npz'  ):
        #L, Tmax, N0, Ts, filename = param;
        self.L = L;
        self.Tmax = Tmax;
        if(N0==[]):
            N0 = np.zeros((2,L,L)); # type, x, y
            N0[0,:int(L/2)-5,:3]+=K; N0[0,int(L/2)+5:,:3]+=K;
            N0[1,int(L/2)-5:int(L/2)+5,:3]+=K; 
            #N0[:,0]+=0.5; N0[:,1]+=0.4; N0[:,2]+=0.3; N0[:,3]+=0.2; N0[:,4]+=0.1;
        self.N = N0;
        if(Ts==0):
            Ts = np.arange(0,Tmax+1,50);
        self.Ts = Ts;
        self.fs=fs;
        self.Ns = np.zeros(( len(Ts), 2, L, L )); self.Ns[0] = N0;
        self.Filename = Filename;
        self.T = 0;
        self.Tc = 0;
        self.dt = dt;
        self.Nshift = np.zeros(Tmax);
        self.Model = model;
        self.K = K;
        #self.Tfix = Tmax;
        #self.Ys = np.zeros((Tmax, 3*L));
        self.Fx = np.zeros((Tmax, L)); # location of front along y-axis (undulation)
        self.Ff = np.zeros((Tmax, L)); # fractions at front along y-axis
        
        
    def generate_step(self, param):
        self.Param = param;
        if(self.Model == 0): # Big mutant
            r, D0, D1, s= param;
            self.r = r; self.D0 = D0; self.D1 = D1; self.s = s; 
        elif(self.Model == 1): # LV model
            r, D, s, a, b = param;
            self.r = r; self.D = D; self.s = s; self.a = a; self.b=b; 
        elif(self.Model == 2): # general convection
            r, Dw, Dm, aw, am = param;
            self.r = r; self.Dw = Dw; self.Dm = Dm; self.aw = aw; self.am=am; 
        elif(self.Model == 3): # growth model
            r, D, s, a, b = param;
            self.r = r; self.D = D; self.s = s; self.a = a; self.b=b; 
            
    def update(self): # kernel input is now (up, left, 0, right, down)
        Nl = np.roll(self.N, -1, axis=2); Nl[:,:,-1] = self.N[:,:,-1]; # Nleft[i,j] is the left-element of N[i,j], i.e. Nleft[i,j] = N[i,j+1] 
        Nr = np.roll(self.N, 1, axis=2); Nr[:,:,0] = self.N[:,:,0];
        Nu = np.roll(self.N, 1, axis=1); Nu[:,0,:] = self.N[:,0,:];
        Nd = np.roll(self.N, -1, axis=1); Nd[:,-1,:] = self.N[:,-1,:];
        
        if(self.Model==0): # Big Mutant
            M = np.copy(self.N); M[0]+=self.N[1]; M[1]+=self.N[0]; # total local population
            Ds = np.zeros((2,self.L,self.L)); Ds = Ds + self.D0;
            Ds = np.where(M==0, Ds, Ds-self.D1*self.N/M ); Ds[0]=Ds[1];
            Dl = np.roll(Ds, -1, axis=2); Dl[:,:,-1] = Ds[:,:,-1]; 
            Dr = np.roll(Ds, 1, axis=2); Dr[:,:,0] = Ds[:,:,0];
            Du = np.roll(Ds, 1, axis=1);
            Dd = np.roll(Ds, -1, axis=1);
            dn = (Dl*Nl + Dr*Nr-4*Ds*self.N + Du*Nu + Dd*Nd) + self.r*self.N; dn[1] = dn[1]+self.r*self.s*self.N[1];
            dn = dn*(1-M/self.K);
            
            return dn
        
        elif(self.Model==1): # LV model
            M = np.copy(self.N); M[0]+=self.N[1]; M[1]+=self.N[0]; # total local population
            f = np.where(M==0, 0, self.N/M); 
            ab = np.zeros((2,self.L,self.L)); ab[0]+=self.a; ab[1]+=self.b;
            r = np.zeros((2,self.L,self.L)); r[0]+=self.r; r[1]+=self.r - self.s;
            dn = self.D*(Nl*Nl + Nr*Nr - 4*self.N*self.N + Nu*Nu + Nd*Nd) + r*(1 - self.N - np.flip(self.N,0)*ab)*self.N;
            
            return dn
        
        elif(self.Model==3): # Growth model
            M = np.copy(self.N); M[0]+=self.N[1]; M[1]+=self.N[0]; # total local population
            f = np.where(M==0, 0, self.N/M); 
            ab = np.zeros((2,self.L,self.L)); ab[0]+=self.a; ab[1]+=self.b;
            r = np.zeros((2,self.L,self.L)); r[0]+=self.r; r[1]+=self.r - self.s;
            dn = self.D*(Nl + Nr - 4*self.N + Nu + Nd) + (r + np.flip(f,0)*ab)*self.N; dn=dn*(1-M/self.K)
        
            return dn
        else:
            print("model not implemented")
    
    def getF(self):
        # This should be implemented in a more clever way in stochastic than in deterministic.
        xs = np.zeros(self.L, dtype=int);
        for i in np.arange(self.L):
            xs[i] = np.max(np.where( self.N[0,i] +self.N[1,i] > 0.5*self.K  )[0]) ;
            #this would be problematic if somehow a row does not have any saturated grid.
        self.Ff[self.T-1] = self.N[1,np.arange(self.L),xs]/(self.N[0,np.arange(self.L),xs] + self.N[1,np.arange(self.L),xs]);
        self.Fx[self.T-1] = xs + self.Nshift[self.T-1];
        
    def proceed(self):
        while(self.T < self.Tmax):
            # New way: instead of looping L^2, use np.roll to update everything at once
            self.Nshift[self.T] = self.Nshift[self.T-1];
            Nnew = self.N + self.update()*self.dt;
            Nnew = Nnew*(Nnew > 0); 
            
            Nsum = Nnew[0] + Nnew[1];
            fw = np.where(Nsum==0, 0, Nnew[0]/Nsum); 
            N = np.random.binomial( self.K, Nsum/self.K, size=(self.L, self.L) );
            Nw = np.random.binomial( N, fw, size=(self.L, self.L) ); Nm = N - Nw;
            self.N[0] = Nw; self.N[1] = Nm;
            
            #it is okay to make shift smaller
            fs = self.fs;
            if(np.max( self.N[0,:,int(self.L/2)] + self.N[1,:,int(self.L/2)] ) > 0.9*self.K   ):
                self.Nshift[self.T] += int(self.L*fs);
                self.N = np.roll(self.N, -int(self.L*fs), axis=2); 
                self.N[:,:,int(self.L)-int(self.L*fs): ] = 0*self.N[:,:,int(self.L)-int(self.L*fs): ];
            self.T = self.T + 1;
            self.getF();
            if(self.T in self.Ts):
                self.Ns[self.Tc, :,:,: ] = self.N;
                self.Tc = self.Tc + 1;
            #if( np.sum( self.Ff[self.T-1] ) > 0.99*L ):  # mutant fix the front
            #    self.Tfix = self.T;
            #    break;
            #elif (np.sum( self.Ff[self.T-1] ) < 0.01*L ): # WT fix the front
            #    self.Tfix = -self.T;
            #    break;
        np.savez( self.Filename, Ns=self.Ns, Ts=self.Ts, Nshift = self.Nshift, Fx = self.Fx, Ff=self.Ff);

In [41]:
#nbi:hide_in
class analysis:
    def __init__(self, sim):
        self.Ns = sim.Ns;
        self.Ts = sim.Ts;
        self.Nshift = sim.Nshift;
        self.Fx = sim.Fx;
        self.Ff = sim.Ff;
        self.L = np.shape(sim.Ns)[2];
        self.Model = sim.Model;
        self.Param = sim.Param;
        self.dt = sim.dt;
        self.T = sim.T;
        self.K = sim.K;
        self.Tmax = sim.Tmax;
    def snap( self, t=-2 ):
        img = np.zeros((np.shape(self.Ns)[2],np.shape(self.Ns)[3],3));
        img[:,:,0]=self.Ns[t,0]; img[:,:,1]=self.Ns[t,1];
        return plt.imshow( img, interpolation='bilinear' )
    def snapsave( self, t=-2 ):
        img = np.zeros((self.L,self.L,3));
        #img[:,:,0]=self.Ns[t,0]; img[:,:,1]=self.Ns[t,1];
        img[:,:,0] = np.where( self.Ns[t,0]+self.Ns[t,1]<0.5*self.K, 0, self.Ns[t,0]/(self.Ns[t,0]+self.Ns[t,1]) );
        img[:,:,1] = np.where( self.Ns[t,0]+self.Ns[t,1]<0.5*self.K, 0, self.Ns[t,1]/(self.Ns[t,0]+self.Ns[t,1]) );
        return img
    def snaps(self, ts, c=True ):
        fig, ax = plt.subplots( 1, len(ts), figsize= (5*len(ts), 7) );
        for i in np.arange(len(ts)):
            img = np.zeros((np.shape(self.Ns)[2],np.shape(self.Ns)[3],3));
            img[:,:,0] = np.where( self.Ns[ts[i],0]+self.Ns[ts[i],1]<0.5*self.K, 0, self.Ns[ts[i],0]/(self.Ns[ts[i],0]+self.Ns[ts[i],1]) );
            img[:,:,1] = np.where( self.Ns[ts[i],0]+self.Ns[ts[i],1]<0.5*self.K, 0, self.Ns[ts[i],1]/(self.Ns[ts[i],0]+self.Ns[ts[i],1]) );
            #img[:,:,3] = self.Ns[ts[i],0]+self.Ns[ts[i],1];
            if(c):
                CS = ax[i].contour( img[:,:,1] );#
                ax[i].clabel(CS, inline=1, fontsize=10);
            ax[i].imshow(img, interpolation='bilinear');
            ax[i].set_title('t='+str( self.Ts[ts[i]+1]*self.dt)[:4]+'hrs');
        fig.suptitle('Model='+str(self.Model)+', params='+str(self.Param), y=0.85, fontsize=15 );
        return fig, ax
    def sector_final(self):
        print('test')
    def sector_traj(self, thres=0.5):
        thres = thres*self.K;
        '''Return x and y coordinates, tracking the sector boundary location at front'''
        yb = np.sum( self.Ff[:self.T,:int(self.L/2)] < thres, axis=1 );
        xb = self.Fx[np.arange(self.T), yb  ];
        yb = self.L/2 - yb;
        return xb, yb
    def fsnap(self, t=-1):
        plt.plot( (a.Fx[t]-np.min(a.Fx[t]) )/(np.max(a.Fx[t])-np.min(a.Fx[t]) )  )
        plt.plot( (a.Ff[t])  )
    def fsnaps(self, ts):
        ts=np.array(ts);
        fig, ax = plt.subplots( 1, 1, figsize= (10, 7) );
        cs=['r','orange','g','cyan','b','purple','k']
        line=[];
        for i in np.arange(len(ts)):
            line+=ax.plot( (a.Fx[ts[i]]-np.min(a.Fx[ts[i]]) )/(np.max(a.Fx[ts[i]])-np.min(a.Fx[ts[i]]) ), color=cs[i]  )
            ax.plot( 1-a.Ff[ts[i]], color=cs[i], ls='--' )
        ax.legend(line, ts*self.dt)
        fig.suptitle('Model='+str(self.Model)+', params='+str(self.Param), y=0.95, fontsize=15 );
        return fig, ax
    def fbt(self):
        '''Return the tracked top/bottom of undulation / genetic wave '''
        fbt = np.zeros((4,self.Tmax));
        for t in np.arange(self.Tmax):
            fbt[0,t] = np.argmin( self.Fx[t] ) # bottom of the undulation wave
            fbt[1,t] = self.L - np.argmax( np.flip(self.Fx[t]) ) # top of the undulation wave
            fbt[2,t] = np.sum( self.Ff[t] < 0.01 )  # bottom of the genetic wave; how mutant invades
            fbt[3,t] = self.L - np.sum( self.Ff[t] > 0.99 ) # top of the genetic wave; how WT invades
        return fbt
    def fhistory(self):
        ''' Show up the history of genetic composition at front like a tapistry (no info on the front shape) '''
        

In [42]:
#nbi:hide_in
def linf(x,a,b):
    return a*x + b;
def vufromf( fbts ):
    ''' Fit genetic wave speed u and expansion speed difference v from fbts data '''
    # Ambiguity in determining the boundary between WT and mutant to determine genetic wave speed: 
    # would pick average of 1% WT and 1% mutant points as a default.
    # Also realized that I do not have good information on v, since I extracted only one coordinate for undulation wave.
    # For now, use Fisher velocity.
    # fix s for each curve. Return as two 2D matrices (us,vs) of size (s x D1)
    us = np.zeros((10,10)); vs = np.zeros((10,10)); 
    for i in np.arange(10):
        for j in np.arange(10):
            d1 = i*0.1; s=j*0.01;
            # as a simplest first implementation, let u be the (total displacement)/(total time)
            thres1 = np.argmin( fbts[i,j,2] ); thres2 = np.argmax( fbts[i,j,2] ); vt = -(fbts[i,j,2,thres1]-fbts[i,j,2,thres2])/(thres1-thres2);
            thres1 = np.argmin( fbts[i,j,3] ); thres2 = np.argmax( fbts[i,j,3] ); vb = -(fbts[i,j,3,thres1]-fbts[i,j,3,thres2])/(thres1-thres2);
            us[i,j] = (vt + vb)/2;
            vs[i,j] = 2*np.sqrt( (1-d1)*(1+s) ) - 2;    
    return (vs,us)
def vufromf2( fbts ):
    ''' Fit genetic wave speed u and expansion speed difference v from fbts data '''
    # Ambiguity in determining the boundary between WT and mutant to determine genetic wave speed: 
    # would pick average of 1% WT and 1% mutant points as a default.
    # Also realized that I do not have good information on v, since I extracted only one coordinate for undulation wave.
    # For now, use Fisher velocity.
    # fix s for each curve. Return as two 2D matrices (us,vs) of size (s x D1)
    us = np.zeros((10,10)); vs = np.zeros((10,10)); 
    for i in np.arange(10):
        for j in np.arange(10):
            d1 = i*0.1; s=j*0.1;
            ## as a simplest first implementation, let u be the (total displacement)/(total time)
            #us[i,j] = (50 - (fbts[i,j,2,-1]+fbts[i,j,3,-1])/2 ) /5000;
            ## second-simplest: average progressively.
            #us[i,j] = np.average ( (50 - (fbts[i,j,2,1:]+fbts[i,j,3,1:])/2 ) /(np.arange(4999)+1) );
            ## third-simplest: average up to boundary-reaching
            thres1 = np.argmin( fbts[i,j,2] ); thres2 = np.argmax( fbts[i,j,2] ); vt = -(fbts[i,j,2,thres1]-fbts[i,j,2,thres2])/(thres1-thres2);
            thres1 = np.argmin( fbts[i,j,3] ); thres2 = np.argmax( fbts[i,j,3] ); vb = -(fbts[i,j,3,thres1]-fbts[i,j,3,thres2])/(thres1-thres2);
            us[i,j] = (vt + vb)/2;
            vs[i,j] = 2*np.sqrt( (1-d1)*(1+s) ) - 2;    
    return (vs,us)